In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Step 1: Imports & Setup
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/mura_modules')

import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

from model_pretrained import build_pretrained_model
from plot_utils import plot_training_history
from config import BATCH_SIZE, IMAGE_SIZE, NUM_CLASSES_BINARY, NUM_CLASSES_BODYPART

In [3]:
# Step 2: Load cached train & validation data
train_data = np.load('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/mura_train_data.npz')
val_data = np.load('/content/drive/MyDrive/Colab Notebooks/Deep_Learning/mura_valid_data.npz')

X_train = train_data['images']
y_bin_train = train_data['labels_binary']
y_part_train = train_data['labels_bodypart']

X_val = val_data['images']
y_bin_val = val_data['labels_binary']
y_part_val = val_data['labels_bodypart']

print(f"Train: {X_train.shape=}, {y_bin_train.shape=}, {y_part_train.shape=}")
print(f"Val:   {X_val.shape=}, {y_bin_val.shape=}, {y_part_val.shape=}")



Train: X_train.shape=(36805, 244, 244, 3), y_bin_train.shape=(36805,), y_part_train.shape=(36805, 1)
Val:   X_val.shape=(3193, 244, 244, 3), y_bin_val.shape=(3193,), y_part_val.shape=(3193, 1)


In [4]:
# Step 3: Build model
model = build_pretrained_model(
    input_shape=IMAGE_SIZE + (3,),
    num_classes_binary=NUM_CLASSES_BINARY,
    num_classes_bodypart=NUM_CLASSES_BODYPART
)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [5]:
# Step 4: Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss={
        'binary_output': 'binary_crossentropy',
        'bodypart_output': 'categorical_crossentropy'
    },
    metrics={
        'binary_output': ['accuracy'],
        'bodypart_output': ['accuracy']
    }
)

In [6]:
# Step 5: Callbacks
callbacks = [
    ModelCheckpoint("pretrained_model_best.keras", monitor="val_loss", save_best_only=True),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)
]

In [ ]:
# Step 6: Fit model
history = model.fit(
    X_train, {"binary_output": y_bin_train, "bodypart_output": y_part_train},
    validation_data=(X_val, {"binary_output": y_bin_val, "bodypart_output": y_part_val}),
    batch_size=BATCH_SIZE,
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Step 7: Evaluate binary task
y_pred_probs = model.predict(X_val)[0]  # binary_output is first output
y_pred = (y_pred_probs > 0.5).astype(int)

print("=== Binary Classification Report (Normal/Abnormal) ===")
print(classification_report(y_bin_val, y_pred, target_names=["Normal", "Abnormal"]))

In [ ]:
# Step 8: Plot training history
plot_training_history(history)